In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tokeniser/bpe_tokenizer_10k_vocab.json
/kaggle/input/bpe-model-with-tinystories/pytorch/default/1/gpt_from_scratch_in_pytorch_on_stories_generation_v2.pth
/kaggle/input/bpe-model-with-tinystories/pytorch/default/1/my_tokenizer.json


In [ ]:
#vocab_size = 4000
#max_len_list = 1119

In [ ]:
!pip install datasets

In [2]:
from datasets import load_dataset

dataset = load_dataset("roneneldan/TinyStories")

README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [2]:
len(dataset["train"]['text'])

2119719

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

In [3]:
dataset = dataset["train"]['text']

In [4]:
dataset = dataset[:300000]

In [5]:
len(dataset)

300000

In [5]:
dataset2 = []
for i in dataset:
    if len(i) > 100:
        dataset2.append(i)

In [6]:
len(dataset2)
dataset2[:2]

['One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.',
 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves that we

In [7]:
!pip install tokenizers

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders

# 1. Make a BPE tokenizer
tokenizer = Tokenizer(models.BPE())

# 2. Pre-tokenizer: ByteLevel is common for GPT-like BPE
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

# 3. Decoder: ByteLevel to decode back
tokenizer.decoder = decoders.ByteLevel()

# 4. Trainer: define vocab size, min frequency, special tokens
trainer = trainers.BpeTrainer(
    vocab_size=4000,  # adjust as needed
    min_frequency=2,
    special_tokens=["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
)

# 5. Train on your list of strings
tokenizer.train_from_iterator(dataset2, trainer)

# 6. Save the tokenizer if you want
tokenizer.save("bpe_tokenizer.json")


In [5]:
from transformers import AutoTokenizer
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders

from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(tokenizer_file="/kaggle/input/tokeniser/bpe_tokenizer_10k_vocab.json")

In [6]:
tokenizer.vocab_size

10000

In [ ]:
encoded = tokenizer.encode(dataset2[0])
encoded

In [ ]:
encoded_dataset = tokenizer(dataset2)

In [ ]:
encoded_dataset

In [8]:
encoded_dataset = []
for sentence in dataset:
    encoded = tokenizer.encode(sentence)
    encoded_dataset.extend(encoded)

In [10]:
print(encoded_dataset[0:2])

[505, 254]


In [ ]:
merged_dataset = []
for sentence in encoded_dataset:
    merged_dataset.extend(sentence)

In [11]:
import torch

class GPTChunkedDataset(torch.utils.data.Dataset):
    def __init__(self, token_ids, block_size):
        self.token_ids = token_ids
        self.block_size = block_size

        # How many full non-overlapping blocks fit?
        self.num_chunks = (len(self.token_ids) - 1) // block_size

    def __len__(self):
        return self.num_chunks

    def __getitem__(self, idx):
        start = idx * self.block_size
        x = torch.tensor(self.token_ids[start : start + self.block_size], dtype=torch.long)
        y = torch.tensor(self.token_ids[start + 1 : start + 1 + self.block_size], dtype=torch.long)
        return x, y


In [12]:
dataset = GPTChunkedDataset(encoded_dataset, block_size=64)

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1024,
    shuffle=True,   # Shuffle for training, optional
    drop_last=True,  # Drop partial batch
    pin_memory=True
)

In [13]:
torch.cuda.device_count()

1

In [ ]:
for xb, yb in loader:
    print(xb.shape)  # (256, 64)     sequence_length=64
    print(yb.shape)  # (256, 64)     sequence_length=64
    break

In [14]:
import torch
import torch.nn as nn

class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, vocab_size=10000, embed_dim=256, max_len=200):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        self.position_emb = nn.Embedding(max_len, embed_dim)

    def forward(self, x):
        batch_size, seq_len = x.size()
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        token_embedding = self.token_emb(x)
        position_embedding = self.position_emb(positions)
        return token_embedding + position_embedding           #torch.Size([256, 64, 128])    b,s,e

In [ ]:
tpe = TokenAndPositionEmbedding(
    vocab_size=4000,
    embed_dim=256,
    max_len=200
)

out = tpe(xb)
out.shape

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim=None, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        self.attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.ln1 = nn.LayerNorm(embed_dim)

        ff_hidden_dim = ff_hidden_dim or embed_dim * 4
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden_dim),
            nn.ReLU(),
            nn.Linear(ff_hidden_dim, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (batch_size, seq_len, embed_dim)
        """
        batch_size, seq_len, _ = x.size()

        # Transpose for MultiheadAttention: (seq_len, batch_size, embed_dim)
        x_t = x.transpose(0, 1)

        # Make causal mask: shape (seq_len, seq_len)
        attn_mask = torch.triu(torch.ones(seq_len, seq_len, device=x.device), diagonal=1).bool()

        # Masked self-attention
        attn_out, _ = self.attn(x_t, x_t, x_t, attn_mask=attn_mask)

        # Residual + LayerNorm
        x2 = self.ln1(x_t + self.dropout(attn_out))

        # Feed Forward
        ff_out = self.ff(x2)

        # Residual + LayerNorm
        out = self.ln2(x2 + self.dropout(ff_out))

        # Transpose back: (batch_size, seq_len, embed_dim)
        return out.transpose(0, 1)

In [16]:
import torch
import torch.nn as nn

class MiniGPTBlock(nn.Module):
    def __init__(self, vocab_size, embed_dim=256, num_heads=8, num_layers=2, max_len=128, dropout=0.1):
        super().__init__()
        self.embedding = TokenAndPositionEmbedding(vocab_size, embed_dim, max_len)
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, dropout=dropout) for _ in range(num_layers)
        ])
        self.ln_f = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        returns logits: (batch_size, seq_len, vocab_size)
        """
        x = self.embedding(x)
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits

In [17]:
model = MiniGPTBlock(vocab_size=10000)

In [18]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [19]:
device

device(type='cuda')

In [20]:
model.to(device)

MiniGPTBlock(
  (embedding): TokenAndPositionEmbedding(
    (token_emb): Embedding(10000, 256)
    (position_emb): Embedding(128, 256)
  )
  (blocks): ModuleList(
    (0-1): 2 x TransformerBlock(
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ReLU()
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
      (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=256, out_features=10000, bias=True)
)

In [ ]:
epochs = 50
learning_rate = 0.002

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in loader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    logits = model(batch_x)  # (batch, seq_len, vocab_size)

    loss = criterion(
      logits.view(-1, vocab_size),
      batch_y.view(-1)
    )

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")


In [24]:
from torch.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR

epochs = 20
model.train()
scaler = GradScaler(device='cuda')
vocab_size = 10000
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, betas=(0.9, 0.98), eps=1e-9)
scheduler = OneCycleLR(optimizer, max_lr=2e-3, total_steps=epochs*len(loader))


for epoch in range(epochs):
    total_loss = 0
    for batch_x, batch_y in loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad(set_to_none=True)
        with autocast(device_type='cuda'):
            logits = model(batch_x)
            loss = criterion(logits.view(-1, vocab_size), batch_y.view(-1))
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        total_loss += loss.item()
    print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")


Epoch: 1, Loss: 4296.2400
Epoch: 2, Loss: 3129.8721
Epoch: 3, Loss: 2720.2601
Epoch: 4, Loss: 2510.4969
Epoch: 5, Loss: 2402.4811
Epoch: 6, Loss: 2335.1439
Epoch: 7, Loss: 2285.8471
Epoch: 8, Loss: 2249.1743
Epoch: 9, Loss: 2219.8334
Epoch: 10, Loss: 2194.9839
Epoch: 11, Loss: 2172.8641
Epoch: 12, Loss: 2152.7044
Epoch: 13, Loss: 2133.6338
Epoch: 14, Loss: 2116.1039
Epoch: 15, Loss: 2099.8382
Epoch: 16, Loss: 2085.4197
Epoch: 17, Loss: 2072.8524
Epoch: 18, Loss: 2062.9057
Epoch: 19, Loss: 2055.7569
Epoch: 20, Loss: 2051.8960


In [33]:
torch.save(model.state_dict(), 'gpt_from_scratch_in_pytorch_on_stories_generation_300k.pth')

In [34]:
import torch

def generate(
    model,
    tokenizer,           # your tokenizer to decode IDs to text
    device,
    start_tokens,        # list of ints, e.g. [BOS] or your prompt encoded
    max_new_tokens=50,   # how many tokens to generate
    block_size=64,       # context window
    temperature=1.0,     # controls randomness
    top_k=None           # optional: restrict to top-k for more randomness control
):
    model.eval()
    generated = start_tokens.copy()
    input_ids = torch.tensor(start_tokens, dtype=torch.long).unsqueeze(0).to(device)  # (1, len)

    for _ in range(max_new_tokens):
        # If context is longer than block_size, crop oldest tokens
        if input_ids.size(1) > block_size:
            input_ids = input_ids[:, -block_size:]

        with torch.no_grad():
            logits = model(input_ids)  # (1, seq_len, vocab_size)

        # Get logits for last token only
        logits = logits[:, -1, :] / temperature  # (1, vocab_size)

        # Optionally top-k filter
        if top_k is not None:
            v, ix = torch.topk(logits, top_k)
            logits[logits < v[0, -1]] = -float('Inf')

        probs = torch.softmax(logits, dim=-1)  # (1, vocab_size)

        next_token = torch.multinomial(probs, num_samples=1)  # (1, 1)

        next_token_id = next_token.item()

        # Append to sequence
        generated.append(next_token_id)

        # Update input_ids
        input_ids = torch.cat([input_ids, next_token], dim=1)  # (1, seq_len+1)


    return generated


In [54]:
# Suppose your tokenizer encodes/decodes:
start_text = "Once upon a time, there was a boy named tim"
start_tokens = tokenizer.encode(start_text) # Or whatever you used

output_tokens = generate(
    model=model,
    tokenizer=tokenizer,
    device=device,
    start_tokens=start_tokens,
    max_new_tokens=250,
    block_size=64,
    temperature=0.4,
    top_k=50  # try top_k for more diverse output
)

# Decode
output_text = tokenizer.decode(output_tokens)
print(output_text)


Once upon a time, there was a boy named timet. He was very curious and loved to explore. One day, he went to the park with his mom. He saw a big tree and wanted to climb it. He asked his mom if he could climb it, but she said no. The boy was sad and he started to cry.

His mom told him that he could not climb the tree. She said that if he was careful, he should not climb the tree.

The boy was sad, but he knew his mom was right. So he decided to try again. He climbed the tree and found a big, juicy apple. He was so happy!

The boy climbed the tree and took a big bite. He was so happy to have found the apple. He ate it all up and was no longer hungry.

The boy was so happy that he had found something so yummy. He ate it up and he felt full and happy. He knew he had made a new friend.Once upon a time there was a little girl named Lucy. She was three years old and loved to play outside. One day, Lucy was playing in the garden when she saw a big, scary dog. The dog was barking and running 

In [32]:
print("done")

done


In [ ]:
tokenizer.save("my_tokenizer.json")